Running a simulation
====================

We will have a look at some of the examples included in `NuRadioMC`. These can be found inside the NuRadioMC installation folder:

In [ ]:
import NuRadioMC
import NuRadioReco
import os

nuradiomc_path = NuRadioMC.__path__[0]
print(f"NuRadioMC is installed at:              {nuradiomc_path}")
print(f"NuRadioMC examples can be found here:   {os.path.join(nuradiomc_path, 'examples')}")
print(f"NuRadioReco examples can be found here: {os.path.join(NuRadioReco.__path__[0], 'examples')}")

1\. Generating neutrinos
----------------------------------
The first step is to generate the incoming neutrinos. We do this using the [`generate_eventlist_cylinder`](https://nu-radio.github.io/NuRadioMC/NuRadioMC/apidoc/NuRadioMC.EvtGen.generator.html#NuRadioMC.EvtGen.generator.generate_eventlist_cylinder) function:

In [ ]:
from NuRadioMC.EvtGen.generator import generate_eventlist_cylinder
from NuRadioReco.utilities import units # the default unit system in NuRadioMC

In [ ]:
# define simulation volume: a cylinder
# with a depth of 3 km and a radius of 4 km
volume = {
'fiducial_zmin':-3 * units.km,
'fiducial_zmax': 0 * units.km,
'fiducial_rmin': 0 * units.km,
'fiducial_rmax': 4 * units.km}

# Now we generate 1000 events with an energy of 1e19 eV
generate_eventlist_cylinder(
    filename='1e19_n1e3.hdf5',
    n_events=1e3,
    Emin=1e19 * units.eV,
    Emax=1e19 * units.eV,
    volume=volume
)

This produces an hdf5 file `1e19_n1e3.hdf5` with 1000 randomly generated events produced by neutrinos with an energy of $10^{19}$ eV.
You can also use a different energy spectrum, change
the neutrino flavour composition, interaction types, 
and many other things - see the [documentation](https://nu-radio.github.io/NuRadioMC/NuRadioMC/pages/Manuals/event_generation.html).


2\. Detector simulation
-----------------------
For most tasks in both simulation and analysis, we use the modules provided by `NuRadioReco`.

Here, we will need the [`RNO_G.hardwareResponseIncorporator`](https://nu-radio.github.io/NuRadioMC/NuRadioReco/apidoc/NuRadioReco.modules.RNO_G.hardwareResponseIncorporator.html)
module to simulate the detector response, and the [`trigger.highLowThreshold`](https://nu-radio.github.io/NuRadioMC/NuRadioReco/apidoc/NuRadioReco.modules.trigger.highLowThreshold.html#NuRadioReco.modules.trigger.highLowThreshold.triggerSimulator) module to simulate the 'trigger'.

In [ ]:
from NuRadioMC.simulation import simulation
import NuRadioReco.modules.RNO_G.hardwareResponseIncorporator
import NuRadioReco.modules.trigger.highLowThreshold

hardware_response = NuRadioReco.modules.RNO_G.hardwareResponseIncorporator.hardwareResponseIncorporator()
highLowThreshold = NuRadioReco.modules.trigger.highLowThreshold.triggerSimulator()

# Some classes are initialized by running `.begin()`.
# In this case, we do not change any parameters, so doing this is optional.
hardware_response.begin()
highLowThreshold.begin()

The simulation is defined in the `simulation` module by the [`simulation`](https://nu-radio.github.io/NuRadioMC/NuRadioMC/apidoc/NuRadioMC.simulation.simulation.html#NuRadioMC.simulation.simulation.simulation) class.
To run our own simulation, we create a simulation class `mySimulation` that inherits from this class.
This allows us to customize as much or as little of the simulation process as we like.
At minimum, however, we should specify the **amplifier response** and the **triggers**.
We do this by defining the `_detector_simulation_filter_amp` and `_detector_simulation_trigger` methods.

In this case, we use the hardware response for RNO-G hardware, and a simple high-low trigger with a threshold of 50 mV (this corresponds to approximately 3 times the RMS noise voltage for the simulated hardware).
To reduce the probability of triggering on pure noise, we use a coincidence condition of 3/4 channels in the phased array (channels 0,1,2 and 3).

In [ ]:
class mySimulation(simulation.simulation):

    def _detector_simulation_filter_amp(self, evt, station, det): # simulate the detector response
        hardware_response.run(evt, station, det, sim_to_data=True)

    def _detector_simulation_trigger(self, evt, station, det): # run the trigger simulation
        highLowThreshold.run(
            evt,
            station,
            det,
            threshold_high=50. * units.mV,
            threshold_low=-50. * units.mV,
            triggered_channels=[0,1,2,3], # run the trigger on channels 0,1,2,3
            coinc_window=30*units.ns,
            number_concidences=3, # require 3 of the channels to satisfy the trigger condition within 30 ns
            trigger_name='main_trigger',
        )

        # you can add additional triggers below, or implement conditional logic to run additional triggers only if the first trigger has fired...


Now we run the simulation! We need to specify the input neutrino file, the detector description, the configuration file, and the output filename(s).
An overview of all the settings that can be specified in the config file, as well as their default values, can be seen in [NuRadioMC/simulation/config_default.yaml](../../simulation/config_default.yaml).

In [ ]:
sim = mySimulation(
    inputfilename="1e19_n1e3.hdf5",
    outputfilename="1e19_n1e3_output.hdf5",
    detectorfile="RNO_G/RNO_single_station.json",
    outputfilenameNuRadioReco="1e19_n1e3_output.nur",
    config_file='config.yaml',
    file_overwrite=True,
    write_detector=False,
    trigger_channels=[0,1,2,3] # specifying all channels used by any of the triggers is optional, but speeds up the simulation process significantly
)

sim.run()

That's it! You now have the output `.hdf5` and `.nur` files - we will see in [the next notebook](W02-reading-nur-files.ipynb) how to read this output.

This example is adapted from `NuRadioMC/examples/01_Veff_simulation` and `NuRadioMC/examples/07_RNO_G_simulation`.
To actually run larger simulations, for example on a cluster, it is generally easier to work with `.py` python scripts as found in those examples, rather than an interactive notebook.
You can have a look at these files, as well as `NuRadioMC/examples/06_webinar` (in which the scripts are very extensively explained).
There is also more information in the [documentation](https://nu-radio.github.io/NuRadioMC/NuRadioMC/pages/manuals.html).